# 04 — Surrogate-guided closed-loop (DL / active learning)

In [ ]:
!pip -q install transformers accelerate biopython pandas numpy matplotlib tqdm scikit-learn pyyaml
import torch, platform
print("torch", torch.__version__, "cuda?", torch.cuda.is_available(), "python", platform.python_version())


In [ ]:
import sys
import os
from pathlib import Path

# Add project root to sys.path so we can import 'src'
current_path = Path(os.getcwd()).resolve()
project_root = None
p = current_path
for _ in range(5):
    if (p / "src").exists() and (p / "src").is_dir():
        project_root = p
        break
    p = p.parent

if project_root and str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Added to sys.path: {project_root}")
elif current_path.name == 'colab':
    # Fallback for Colab default structure if cloning repo
    # If we are in /content/repo/colab, parent is repo
    parent = str(current_path.parent)
    if parent not in sys.path:
        sys.path.append(parent)
        print(f"Added to sys.path: {parent}")


In [ ]:
!git clone -q https://github.com/dauparas/ProteinMPNN.git
!pip -q install -r ProteinMPNN/requirements.txt


In [ ]:
from pathlib import Path
import pandas as pd
import torch
from src.data.scaffolds import load_scaffold
from src.generate.proteinmpnn import run_proteinmpnn, read_fasta_sequences
from src.evaluate.esmfold_eval import evaluate_batch
from src.generate.mutations import make_mutant_pool
from src.models.surrogate import SurrogateConfig, get_embeddings, train_surrogate, predict_surrogate

OUT = Path("results")
sc = load_scaffold("1AKL","A", OUT/"scaffolds")

# labeled set
fasta = run_proteinmpnn(sc.pdb_path, OUT/"mpnn_surrogate_r0", Path("ProteinMPNN"), num_seqs=60, sampling_temp=0.2, seed=42)
seqs0 = read_fasta_sequences(fasta)[:40]
lab = evaluate_batch(seqs0, model_id="facebook/esmfold_v1", device="cuda", out_dir=OUT/"pdb"/"surrogate_r0", max_n=20)

train_seqs = [r.sequence for r in lab]
y = torch.tensor([r.mean_plddt for r in lab], dtype=torch.float32)

cfg = SurrogateConfig(esm2_model_id="facebook/esm2_t12_35M_UR50D", pool="mean", epochs=8, lr=1e-3, device="cuda")
X = get_embeddings(train_seqs, cfg.esm2_model_id, cfg.pool, cfg.device)
m = train_surrogate(X, y, cfg)

# propose many, fold few
cands = make_mutant_pool(train_seqs, n=200, rate=0.03, seed=123)
Xc = get_embeddings(cands, cfg.esm2_model_id, cfg.pool, cfg.device)
pred = predict_surrogate(m, Xc, cfg.device).numpy()
top_idx = pred.argsort()[::-1][:30]
to_fold = [cands[i] for i in top_idx]
res = evaluate_batch(to_fold, model_id="facebook/esmfold_v1", device="cuda", out_dir=OUT/"pdb"/"surrogate_r1", max_n=20)

df = pd.DataFrame([{"sequence":r.sequence, "mean_plddt":r.mean_plddt} for r in res]).sort_values("mean_plddt", ascending=False)
df.head(10)


In [ ]:
from pathlib import Path
OUT = Path("results")
(OUT/"tables").mkdir(parents=True, exist_ok=True)
df.to_csv(OUT/"tables"/"surrogate_guided.csv", index=False)
print("saved", OUT/"tables"/"surrogate_guided.csv")

# Save results to GitHub (optional)
try:
    from src.utils.github_save import save_results_to_github
    save_results_to_github("exp04_surrogate_guided")
except Exception as e:
    print(f"Note: GitHub save skipped ({e})")
